#Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Put Data in DataFrame (Articles)

In [ ]:
import pandas as pd
import glob, os

# Place dataset path here
FNNpath = "../Datasets/FNN Titles/"
CoAIDpath = "../Datasets/CoAID/"

politiFactFake = pd.read_csv(FNNpath+"politifact_fake.csv", usecols=['title'])
politiFactFake['label']=0
gossipCopFake = pd.read_csv(FNNpath+"gossipcop_fake.csv", usecols=['title'])
gossipCopFake['label']=0
politiFactTrue = pd.read_csv(FNNpath+"politifact_real.csv", usecols=['title'], nrows=len(politiFactFake.values))
politiFactTrue['label']=1
gossipCopTrue = pd.read_csv(FNNpath+"gossipcop_real.csv", usecols=['title'],  nrows=len(gossipCopFake.values))
gossipCopTrue['label']=1

CoAIDFalse = pd.read_csv(CoAIDpath+"NewsFakeCOVID-19.csv", usecols=['title'])
CoAIDFalse['label']=0
CoAIDTrue = pd.read_csv(CoAIDpath+"NewsRealCOVID-19.csv", usecols=['title'], nrows=len(CoAIDFalse.values))
CoAIDTrue['label']=1


dfTotal = pd.concat([politiFactTrue, gossipCopTrue, politiFactFake, gossipCopFake, CoAIDTrue, CoAIDFalse])


X = dfTotal['title'].values
y = dfTotal['label'].values
print("len(X)", len(X))
print("len(y)", len(y))

len(X) 12654
len(y) 12654


# Finetune Fake News Detection Model on CoAID&FNN Dataset

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 140 kB 47.0 MB/s 
     |████████████████████████████████| 212 kB 54.3 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 33.2 MB/s 
     |████████████████████████████████| 127 kB 51.1 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 54.5 MB/s 
     |████████████████████████████████| 271 kB 52.4 MB/s 
     |████████████████████████████████| 112 kB 55.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing ins

In [ ]:
from sklearn.utils import shuffle
dfTotal = shuffle(dfTotal)
from datasets import Dataset
train_dataset = Dataset.from_pandas(dfTotal)

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/735 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:


def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')


train_dataset = train_dataset.map(tokenize_data, batched=True)


from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)


from transformers import Trainer

trainer_train = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
)


  0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
trainer_train.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title, __index_level_0__. If title, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12654
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4746


Step,Training Loss
500,0.601900
1000,0.522300
1500,0.501300
2000,0.416600
2500,0.423900
3000,0.402000
3500,0.337800
4000,0.303700
4500,0.311600


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=4746, training_loss=0.41877616210371643, metrics={'train_runtime': 2245.7505, 'train_samples_per_second': 16.904, 'train_steps_per_second': 2.113, 'total_flos': 9988221883576320.0, 'train_loss': 0.41877616210371643, 'epoch': 3.0})

In [ ]:
import torch
model_path = '../models/Fake-News-Bert-Detect_CoAID&FNN.pt'
torch.save(model.state_dict(),model_path)

In [ ]:
path_external = "/content/drive/MyDrive/CoVerifi&MedVerifi/" # Replace with appropriate path
dfTotal_external = pd.read_csv(path_external+"Combined_News.csv", usecols=['title', 'label'])

def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0
dfTotal_external = dfTotal_external.dropna()
dfTotal_external['label'] = dfTotal_external['label'].apply(lambda label: binaryLabel(str(label)))
dataset_external = Dataset.from_pandas(dfTotal_external)

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
dataset_external = dataset_external.map(tokenize_data, batched=True)
trainer_eval = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title, __index_level_0__. If title, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.9025081479382173,
 'eval_loss': 0.4031279981136322,
 'eval_runtime': 132.0646,
 'eval_samples_per_second': 53.436,
 'eval_steps_per_second': 6.686}